In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import gym

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras.optimizers import Adam

In [3]:
env = gym.make('CartPole-v1', render_mode='human')

In [4]:
# Number of observations 
no_observations=env.observation_space.shape[0]

In [5]:
# umber of actions can take and those are continous / discrete  identifed
no_actions= env.action_space.n

In [6]:
no_actions

2

In [7]:
# When creating neural network number of observations == input neurons number and number of neurons on final layer == actions

# Traiing Model network - Value Network

model = Sequential()
#Add layers
model.add(Dense(4, input_shape=(1,no_observations)))
model.add(Activation('relu'))

#  Input shape must be provided defined in first layer to compile properly but can define at any layer
model.add(Dense(4))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))



# output or last layer gives number of actions
model.add(Dense(2))
model.add(Activation('linear'))

In [8]:
# print(model.summary())

In [9]:
from tensorflow.keras.models import clone_model

In [10]:
# model copying 
target_model = clone_model(model)

In [11]:
# Network ouline defined - But before training Configure Hyperparametrs such as

learning_parameter = 0.001
Epochs = 500  # number of iterations 
epsilon = 1.0 # How fast will learn 1 -> max but later modify according to learning with accracy
epsilon_reduce = 0.995 # 99.5% reduced to previous epsilon
gamma = 0.95

# Epsilon - greedy => Exploration vs Exploitaction to select / predict a action value
# So required are a model, observation and epsilon



In [12]:
def greedy_epsilon_action(model,observation,epsilon):
    
    # Intialy not aware which action to perform so make a random action and then learn
    if np.random.random() > epsilon:
        predictions = model.predict(observation,verbose =0)
        action = np.argmax(predictions)  # picking max value to perform aaction
        
    else:
        #Explore
        action = np.random.randint(0, env.action_space.n) # choosing randomly an action b/w 0 to max actions can perform on env
        
    return action

In [13]:
# from collections import deque

# deque(maxlen=5)

In [14]:
# d= deque(maxlen=5)

# for i in range(10):
#     d.append(i)
#     print(d)

In [15]:

from collections import deque

# for replay buffer - Need - window / repay buffer [Deque obj] size to store , batch size, model, target model

replay_buffer = deque(maxlen=20000)

In [16]:
# Replay - Pick buffer size / sequence of frames/data to hold 
# use deque from collections 
import random


def replay(buffer,batch_size,model,target_model):
    
    if len(buffer) < batch_size:
        return # No action untill buffer filled
    samples = random.sample(buffer,batch_size)   # select ramdomly batch of values from buffer
    
    target_batch = []
    
    # samples -> collection of buffers ,1 buffer = 1 observations, observation -> state,reward,action, new_state,done_flag
    zipped_samples = list(zip(*samples))      #tuple of 
    states, actions ,rewards,new_states,dones = zipped_samples
    
    targets = target_model.predict(np.array(states),verbose =0)
    
    q_values = model.predict(np.array(new_states),verbose=0).flatten()
    
    for i in range(batch_size):
        
        q_value = q_values[i]
        target = targets[i].copy()
        
        if dones[i] :
            target[0][actions[i]] = rewards[i]
            
        else:
            target[0][actions[i]] = rewards[i] + q_value*gamma
            
        target_batch.append(target)
        
    model.fit(np.array(states),np.array(target_batch),epochs =1, verbose= 0)
    
    

In [17]:
update_target_model = 10

def update_model_handler(ephocs, update_target_model, model,target_model):
    if ephocs >0 and ephocs % update_target_model == 0:
        target_model.set_weights(model.get_weights())

In [18]:
model.compile(loss='mse',optimizer=Adam(learning_rate=learning_parameter))

In [19]:
# model.summary()


In [21]:
score = 0

for ephocs in range(Epochs):
    observation = env.reset()[0]
    observation = observation.reshape([1,4])
    done = False
    
    point = 0 
    while not done:
        
        point += 1
        action = greedy_epsilon_action(model,observation,epsilon)
        next_observation,reward,done,truncate,info = env.step(action)
        next_observation = next_observation.reshape([1,4])
        
        replay_buffer.append((observation,action,reward,next_observation,done))
        observation = next_observation
        
        replay(replay_buffer,32,model,target_model)
    
    epsilon *= epsilon_reduce
    
    update_model_handler(ephocs, update_target_model, model,target_model)
    
    if point > score:
        score = point
        
    if ephocs % 10 == 0:
        print(f'score: {score} at ephoc:{ephocs} when epsilon: {epsilon}')
        
    
    
    


score: 15 at ephoc:0 when epsilon: 0.9000874278732445
score: 34 at ephoc:10 when epsilon: 0.8560822709551227
score: 42 at ephoc:20 when epsilon: 0.8142285204175609
score: 42 at ephoc:30 when epsilon: 0.7744209942832988
score: 48 at ephoc:40 when epsilon: 0.736559652908221
score: 48 at ephoc:50 when epsilon: 0.7005493475733617
score: 48 at ephoc:60 when epsilon: 0.6662995813682115
score: 48 at ephoc:70 when epsilon: 0.6337242817644086
score: 48 at ephoc:80 when epsilon: 0.6027415843082742
score: 48 at ephoc:90 when epsilon: 0.5732736268885887
score: 48 at ephoc:100 when epsilon: 0.5452463540625918
score: 48 at ephoc:110 when epsilon: 0.5185893309484582
score: 68 at ephoc:120 when epsilon: 0.4932355662165453
score: 68 at ephoc:130 when epsilon: 0.46912134373457726
score: 68 at ephoc:140 when epsilon: 0.446186062443672
score: 68 at ephoc:150 when epsilon: 0.42437208406280985
score: 68 at ephoc:160 when epsilon: 0.4036245882390106
score: 68 at ephoc:170 when epsilon: 0.38389143477919885
sc

In [29]:
observation = env.reset()[0]
observation

array([ 0.01554028,  0.02787708,  0.02047598, -0.0440359 ], dtype=float32)

In [34]:
observation = env.reset()[0]
        #For debbuging we use render
for i in range(3000):
    env.render()
    action = np.argmax(model.predict(observation.reshape([1,4]),verbose=0))
    observation,reward,done,trancate,info = env.step(action)
    observation = observation.reshape([1,4])
            
    if done:
        print(i)
        print(observation,reward,done,trancate,info)
        break
env.close()

8
[[-0.18335414 -1.7350829   0.22137108  2.7829535 ]] 1.0 True False {}


In [33]:
env = gym.make('CartPole-v1', render_mode='human')